In [1]:
!pip install python-dotenv gradio openai cassandra-driver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0

In [2]:
import os
import io
import IPython.display
from PIL import Image
import base64
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

# set parameters for AstraDB
astradb_token= os.environ['ASTRA_TOKEN']
astradb_secure_bundle_path= os.environ['SECURE_CONNECT_BUNDLE_PATH']
api_key = os.environ['OPENAI_API_KEY']

In [ ]:
cluster = Cluster(
    cloud={
        "secure_connect_bundle": astradb_secure_bundle_path,
    },
    auth_provider=PlainTextAuthProvider(
        "token", astradb_token
    ),
)

session = cluster.connect()
session.execute("use vector;")

In [ ]:
import openai
openai.api_key = api_key
def generate_embedding(text):
    model = "text-embedding-ada-002"
    response = openai.Embedding.create(model=model, input=text)
    return response.data[0]['embedding']

In [3]:
def get_completion_from_messages(messages, model="gpt-4", temperature=0):
    messages =  [
                  {'role':'system', 'content':f'You are a chatbot for giving recommendations for the best practices in data modelling in Cassandra and you provide best practices recommendations and you provide a CQL with the best possible partition key for their table in the prompt [{messages}].'},
                  {'role':'system', 'content':'You need to ask questions related to how many rows do they expect in a partition and an initial draft for their table and you will calculate the partition size'},
                  {'role':'system', 'content': "Here an example how to calculate the partition size: CREATE TABLE temperature_readings (device_id UUID,timestamp TIMESTAMP,temperature FLOAT,PRIMARY KEY (device_id, timestamp)); "},
                  {'role':'system', 'content': f'This is an example, you need to use the new values in the [{messages}].They will have 500K rows for each device per day: Using the  given data type size you can calculate the partition size:Partition size = (column1 type size + column2 type size + column3 type size) x Number of rows= (16+8+4)*500000=  14000000 bytes = 13MB'},
                  {'role':'system', 'content': f'You need to check the partition key in the table definition in [{messages}] and calculate the partition size for each table depending on the row count. One of the best practice is to have a partition size less than 100MB and if it is above that you can recommend using another column in the partition key or add time bucketing.'},
                  {'role':'system', 'content': "And you can also provide other best practices for Cassandra Data Modelling like using TTL , using a new denormalized table for each query access instead of using Secondary Index and Materialized Views"},
              ]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [4]:
import gradio as gr

def image_to_base64_str(pil_image):
    byte_arr = io.BytesIO()
    pil_image.save(byte_arr, format='PNG')
    byte_arr = byte_arr.getvalue()
    return str(base64.b64encode(byte_arr).decode('utf-8'))

def datamodel(prompt):
    #base64_image = image_to_base64_str(image)
    result = get_completion_from_messages(prompt)
    return result

gr.close_all()
demo = gr.Interface(fn=datamodel,
                    inputs=[gr.Textbox(label="Can you give an explanation about your data model and initial table design please?") ],
                    outputs=[gr.Textbox(label="Recommendation") ],
                    title="Data Model Agent",
                    description="Data Model Agent",
                    allow_flagging="never")

demo.launch(share=True, server_port=1788)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9120dae85491e3cb99.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
gr.close_all()

Closing server running on port: 1788
